# Exemplo 05: Redução de Dimensionalidade
## Elementos nutricionais mais importantes

*Redução de dimensionalidade* ou *redução de dimensão* é uma técnica usada em mineração de dados e aprendizado de máquina que tem como objetivo a redução do número de variáveis de entrada para extrair um conjunto de variáveis principais. Apesar do risco de perda de acurácia na análise, a redução do tempo de processamento torna esta técnca importante para a análise de dados multivariáveis. 

Os algoritmos de seleção de atributos tentam encontrar um subconjunto das variáveis de entrada (também chamadas de atributos) mais importante para a análise. A análise de dados como regressão ou classificação pode ser feita no espaço reduzido com mais velocidade e precisão do que no espaço original.

## Principal Component Analysis (PCA)

A principal técnica linear para redução de dimensionalidade é a análise de componentes principais (PCA). PCA realiza um mapeamento linear dos dados de entrada para um espaço de menor dimensão, de forma que a variação dos dados na representação de dimensão menor seja maximizada. Na prática, a matriz de covariância (e às vezes a de correlação) dos dados é construída e são calculados os autovetores nessa matriz. Os vetores próprios que correspondem aos maiores valores próprios (os componentes principais) que agora podem ser usados para reconstruir uma grande parte da variação dos dados originais. Além disso, os primeiros autovetores podem ser interpretados como termos do comportamento físico do sistema, porque geralmente contribuem com a maior parte da energia do sistema. Uma redução usando o PCA deve ser comprovada caso a caso, pois nem todos os sistemas exibem esse comportamento de equivalencia. O espaço original foi reduzido com perda de dados, mas esperamos manter a variação mais importante para a análise do problema.

## Exemplo

Este exemplo apresenta a redução de dimensionalidade usando PCA sobre dados nutricionais de alimentos da USDA National Nutrient Database com 45 campos. O objetivo é obter relações dos alimentos através apenas dos principais componente. 


In [1]:
# Find Spark executable
import findspark
findspark.init()

## Carregando de bibliotecas 

In [2]:
# Load libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA

from pyspark.ml.linalg import Vectors

import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt

import time
start_time = time.time()

%matplotlib inline

## Funções auxiliares

In [3]:
def sliceFeatures(df, nValores, nomeSaidas, nomeEntrada):#dataframe / numero de colunas a serem criadas / nome das colunas de saida / ...coluna de entrada
    for i in range(nValores):
        slicer = VectorSlicer(inputCol = nomeEntrada, outputCol = nomeSaidas + str(i+1), indices=[i])
        df = slicer.transform(df)
    return df

def returnVector(df, name):#dataframe / nome da coluna , retorna um vetor da coluna especificada
    aux = np.array(df.select(name).collect())
    aux2 = []
    for i in range(df.count()):
        aux2.append(float(aux[i]))
    return aux2

def returnDimesion(nAmostras, nTipos, rangeDaMedia):#retorna um vetor np com nAmostras, com nTipos com a mesma media, que varia de 1 a rangeDaMedia 
    d = []
    for k in range(nTipo):
        t = np.random.poisson(lam=rd.randrange(1, rangeDaMedia), size = nAmostras)
        d = np.concatenate((d, t))
    return d   

## Parameters configuration

In [4]:
# Path to dataset file
data_path='./data/'

# Number of principal components
nComponentes=5

## Creating Spark environment

In [5]:
# Create Spark Session
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("ReducaoDimensionalidadePCA") \
        .getOrCreate()

## Reading Data

In [6]:
# Define Dataframe schema
food_schema = StructType([
    StructField('ID', IntegerType()),
    StructField('FoodGroup', StringType(), True),
    StructField('ShortDescrip', StringType(), True),
    StructField('Descrip', StringType(), True),
    StructField('CommonName', StringType(), True),
    StructField('MfgName', StringType(), True),
    StructField('ScientificName', StringType(), True),
    StructField('Energy_kcal', FloatType(), True),
    StructField('Protein_g', FloatType(), True),
    StructField('Fat_g', FloatType(), True),
    StructField('Carb_g', FloatType(), True),
    StructField('Sugar_g', FloatType(), True),
    StructField('Fiber_g', FloatType(), True),
    StructField('VitA_mcg', FloatType(), True),
    StructField('VitB6_mg', FloatType(), True),
    StructField('VitB12_mcg', FloatType(), True),
    StructField('VitC_mg', FloatType(), True),
    StructField('VitE_mg', FloatType(), True),
    StructField('Folate_mcg', FloatType(), True),
    StructField('Niacin_mg', FloatType(), True),
    StructField('Riboflavin_mg', FloatType(), True),
    StructField('Thiamin_mg', FloatType(), True),
    StructField('Calcium_mg', FloatType(), True),
    StructField('Copper_mcg', FloatType(), True),
    StructField('Iron_mg', FloatType(), True),
    StructField('Magnesium_mg', FloatType(), True),
    StructField('Manganese_mg', FloatType(), True),
    StructField('Phosphorus_mg', FloatType(), True),
    StructField('Selenium_mcg', FloatType(), True),
    StructField('Zinc_mg', FloatType(), True),
    StructField('VitA_USRDA', FloatType(), True),
    StructField('VitB6_USRDA', FloatType(), True),
    StructField('VitB12_USRDA', FloatType(), True),
    StructField('VitC_USRDA', FloatType(), True),
    StructField('VitE_USRDA', FloatType(), True),
    StructField('Folate_USRDA', FloatType(), True),
    StructField('Niacin_USRDA', FloatType(), True),
    StructField('Riboflavin_USRDA', FloatType(), True),
    StructField('Thiamin_USRDA', FloatType(), True),
    StructField('Calcium_USRDA', FloatType(), True),
    StructField('Copper_USRDA', FloatType(), True),
    StructField('Magnesium_USRDA', FloatType(), True),
    StructField('Phosphorus_USRDA', FloatType(), True),
    StructField('Selenium_USRDA', FloatType(), True),
    StructField('Zinc_USRDA', FloatType(), True),
])

#Extrai dados de dados dos alimentos CSV para Dataframe Spark
food = spark.read.format("csv").options(header='true'). \
         load(data_path+"nndb_flat.csv.gz",schema=food_schema)

#food.dtypes
food.show(5)

+----+--------------------+--------------------+--------------------+----------+-------+--------------+-----------+---------+-----+------+-------+-------+--------+--------+----------+-------+-------+----------+---------+-------------+----------+----------+----------+-------+------------+------------+-------------+------------+-------+----------+------------+------------+----------+-----------+------------+------------+----------------+-------------+-------------+------------+---------------+----------------+--------------+------------+
|  ID|           FoodGroup|        ShortDescrip|             Descrip|CommonName|MfgName|ScientificName|Energy_kcal|Protein_g|Fat_g|Carb_g|Sugar_g|Fiber_g|VitA_mcg|VitB6_mg|VitB12_mcg|VitC_mg|VitE_mg|Folate_mcg|Niacin_mg|Riboflavin_mg|Thiamin_mg|Calcium_mg|Copper_mcg|Iron_mg|Magnesium_mg|Manganese_mg|Phosphorus_mg|Selenium_mcg|Zinc_mg|VitA_USRDA| VitB6_USRDA|VitB12_USRDA|VitC_USRDA| VitE_USRDA|Folate_USRDA|Niacin_USRDA|Riboflavin_USRDA|Thiamin_USRDA|Calci

In [7]:
# The feature column should join all parameters as a Vector
# Set the column names that is not part of features list
ignore = ['ID', 'FoodGroup', 'ShortDescrip', 'Descrip', 'CommonName', 'MfgName', 'ScientificName']

# list will be the value of all columns parts of features vector. Save it, it will be used to revert the vector
list = [x for x in food.columns if x not in ignore]

# VectorAssembler mount the vector of features
assembler = VectorAssembler(
            inputCols=list,
            outputCol='features')

# Create final dataframe composed by label and a column of features vector
# To avoid errors invalid collumn fields shoud be skiped
data = (assembler.setHandleInvalid("skip").transform(food).select("ID","Descrip","FoodGroup","features"))

print("Final Dataframe")
data.printSchema()

Final Dataframe
root
 |-- ID: integer (nullable = true)
 |-- Descrip: string (nullable = true)
 |-- FoodGroup: string (nullable = true)
 |-- features: vector (nullable = true)



In [8]:
# If feature vector use different scale, all collumn should be normalized 
standardScaler = StandardScaler(inputCol="features", outputCol="scalled_features", withStd=True, withMean=True)
scalledmodel = standardScaler.fit(data)

# Remove unscalled feature collumn
scalled_data = scalledmodel.transform(data).drop('features')

scalled_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Descrip: string (nullable = true)
 |-- FoodGroup: string (nullable = true)
 |-- scalled_features: vector (nullable = true)



In [9]:
# Rum PCA algorithm
# k= number of components to be reduced. PCA find the k most important components
pca = PCA(k=nComponentes, inputCol="scalled_features", outputCol="pca_features")
pcamodel = pca.fit(scalled_data)
result = pcamodel.transform(scalled_data)

result.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Descrip: string (nullable = true)
 |-- FoodGroup: string (nullable = true)
 |-- scalled_features: vector (nullable = true)
 |-- pca_features: vector (nullable = true)



In [10]:
# Show the importance of each component: explainedVariance

pca_var= np.round(100.00*pcamodel.explainedVariance.toArray(),nComponentes)

print("Relevance of each component")
print(pca_var)

Relevance of each component
[26.62088 10.77726  9.14715  6.78869  6.22337]


In [11]:
# List the relevance of each components by element
pcs = np.round(pcamodel.pc.toArray(),4)
pca_comp = pd.DataFrame(pcs, columns = ['PC1','PC2','PC3','PC4','PC5'], index = list)
print(pca_comp)

                     PC1     PC2     PC3     PC4     PC5
Energy_kcal      -0.0950  0.0971 -0.1430  0.2684 -0.2971
Protein_g        -0.1136 -0.0845 -0.1682 -0.3024 -0.1210
Fat_g            -0.0137  0.0168 -0.1343  0.2253 -0.3766
Carb_g           -0.1080  0.1775 -0.0080  0.2643  0.0814
Sugar_g          -0.0429  0.1129  0.0397  0.2547  0.0580
Fiber_g          -0.1164  0.1168 -0.1385  0.1861  0.0795
VitA_mcg         -0.1069 -0.3627  0.0678  0.1845  0.0942
VitB6_mg         -0.2389  0.0947  0.1247 -0.0042  0.0095
VitB12_mcg       -0.1437 -0.3618  0.0702  0.0029 -0.0170
VitC_mg          -0.0699  0.0406  0.0866  0.1117  0.3092
VitE_mg          -0.0997  0.0674 -0.0455  0.2718 -0.3437
Folate_mcg       -0.2127  0.1207  0.1648 -0.0021 -0.0179
Niacin_mg        -0.2569  0.0731  0.1460 -0.1217 -0.0322
Riboflavin_mg    -0.2590 -0.0199  0.1611 -0.0508  0.0502
Thiamin_mg       -0.2047  0.1314  0.1419 -0.0947 -0.0345
Calcium_mg       -0.1201  0.0886 -0.2972  0.0138  0.2579
Copper_mcg       -0.1393 -0.328

In [12]:
# List PCA feature for each food 
result_pca = result.select('Descrip',"pca_features")

result_pca.show(truncate=False)

+------------------------------------------------------------+------------------------------------------------------------------------------------------------------+
|Descrip                                                     |pca_features                                                                                          |
+------------------------------------------------------------+------------------------------------------------------------------------------------------------------+
|Butter, salted                                              |[1.6961589673245903,-0.40130641575715076,-0.3119835907890767,2.443965041600696,-2.6857802271447206]   |
|Butter, whipped, with salt                                  |[1.6925717792213761,-0.4106487760502162,-0.31478301514174356,2.4554947024364115,-2.6812701865908326]  |
|Butter oil, anhydrous                                       |[1.673611549674143,-0.4089146295578469,-0.4426331554418034,3.136411283933793,-3.4605379106257987]     |
|Che

In [13]:
spark.stop()
print("--- Execution time: %s seconds ---" % (time.time() - start_time))

--- Execution time: 14.05980110168457 seconds ---
